<a href="https://colab.research.google.com/github/lSelectral/shortest-route/blob/master/SHORTHEST_PATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DEMO

[ALTERNTİF BANK IDEATHON - ROUTE OPTIMISATION](https://alternatif-bank-shortest-route.tiiny.site/)


> Also you can run see working version by running yourself. Just do below steps

### HOW TO RUN

1. Run the **`!pip install osmnx`**
2. After it finishes, click the **`Restart Runtime`** button
3. Then run the next cell.
4. Run **`DIJKSTRA LIBRARY`** cell.
5. Run **`ADD COORDINATE DATA`** cell.
6. Run **`GLOBAL VARIABLE`** cell.
7. Run **`FUNCTIONS`** cell.
8. Run **`IMPLEMENT`** cells. (Only last cell is required)


---


> MODIFICATION

You can change coordinate data in `GLOBAL VARIABLE > coordinate_data` variable.
After that run `GLOBAL_VARIABLE` and `IMPLEMENTATION` cells.


---


> MADE FOR ALTERNATİF BANK ENGELSİZ BANKACILIK IDEATHON

### IMPORT PACKAGES

In [ ]:
!pip install osmnx

In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd
import folium
import copy
import warnings
warnings.filterwarnings('ignore')

### DIJKSTRA LIBRARY

In [ ]:
from queue import PriorityQueue

class Graph:
    def __init__(self, num_of_vertices):
        self.v = num_of_vertices
        self.edges = [[-1 for i in range(num_of_vertices)] for j in range(num_of_vertices)]
        self.visited = []
    def add_edge(self, u, v, weight):
        self.edges[u][v] = weight
        self.edges[v][u] = weight

        
def dijkstra(graph, start_vertex):
    D = {v:float('inf') for v in range(graph.v)}
    D[start_vertex] = 0

    pq = PriorityQueue()
    pq.put((0, start_vertex))

    while not pq.empty():
        (dist, current_vertex) = pq.get()
        graph.visited.append(current_vertex)

        for neighbor in range(graph.v):
            if graph.edges[current_vertex][neighbor] != -1:
                distance = graph.edges[current_vertex][neighbor]
                if neighbor not in graph.visited:
                    old_cost = D[neighbor]
                    new_cost = D[current_vertex] + distance
                    if new_cost < old_cost:
                        pq.put((new_cost, neighbor))
                        D[neighbor] = new_cost
    return D

### ADD COORDINATE DATA

In [ ]:
points = pd.DataFrame({'lat':[
          37.847056,
          37.848606,
          37.845784,
          37.845458,
          37.847571 
          ],
          "lon":[27.849861,
                 27.847391,
                 27.849596,
                 27.852334,
                 27.854899
          ]}, dtype="float")

In [ ]:
points_2 = pd.DataFrame({"lat":[
                              37.837826,  
                              37.839538,     
                              37.838734,
                              37.840225,
                              37.837253
                        ],
                        "lon":[
                                27.855620,
                                27.851146,
                                27.857077,
                                27.857664,
                                27.858006
                        ]}, dtype="float")

### GLOBAL VARIABLE

In [ ]:
# COORDINATES TO USE
coordinate_data = points

In [ ]:
# GRAPH TO USE
def coord_tuple_by_index(ind):
    return (coordinate_data.iloc[ind]["lat"], coordinate_data.iloc[ind]["lon"])
# graph = ox.graph_from_place('Efeler, Aydın, Türkiye', network_type='drive', simplify=True)
graph = ox.graph_from_point(coord_tuple_by_index(0), network_type="drive", dist_type="network")

In [ ]:
color_list = [
"#FFBF00",
"#DFFF00",
"#FF7F50",
"#DE3163",
"#9FE2BF",
"#40E0D0",
"#6495ED",
"#CCCCFF",
"#00FF00",
"#FF00FF",
"#C1FFC1",
"#FFD700",
"#EE9A00"
]

### FUNCTIONS

In [ ]:
def get_distance_between_two_point(x1,x2):
    orig_node = ox.get_nearest_node(graph, x1)
    target_node = ox.get_nearest_node(graph, x2)
    return nx.shortest_path_length(G=graph, source=orig_node, target=target_node, weight='length')

In [ ]:
def get_vertexes():
    l = []
    for i in range(len(coordinate_data)):
        tempList = []
        for j in range(len(coordinate_data)-1):
            temp = coordinate_data.drop(coordinate_data.index[i])
            e = get_distance_between_two_point(x1=(coordinate_data.iloc[i]['lat'],coordinate_data.iloc[i]['lon']), x2=(temp.iloc[j]['lat'],temp.iloc[j]['lon']))
            tempList.append(e)
        l.append(tempList)
    return l

In [ ]:
def get_neighbors_of_indexes():
    distance_data = get_vertexes()
    indexList = []
    for i in distance_data:
        temp = copy.deepcopy(i)
        temp.sort()
        smallest_distance_list = []
        smallest_distance_list.append(i.index(temp[0]))
        smallest_distance_list.append(i.index(temp[1]))
        indexList.append(smallest_distance_list)
    return indexList

In [ ]:
def get_shortest_route(startPoint, arrivalPoint, optimizer = "time"):
    origin_node = ox.get_nearest_node(graph, startPoint)
    destination_node = ox.get_nearest_node(graph, arrivalPoint)
    return nx.shortest_path(graph, origin_node, destination_node, weight = optimizer)

In [ ]:
def get_shortest_routes():
    routes = []
    points = coordinate_data
    noi = get_neighbors_of_indexes()
    for i in range(len(points)):
        j = noi[i]
        a = []
        if (i == 0):
            a = get_shortest_route((points.iloc[i]["lat"], points.iloc[i]["lon"]), (points.iloc[j[0]]["lat"], points.iloc[j[0]]["lon"]))
        b = get_shortest_route((points.iloc[i]["lat"], points.iloc[i]["lon"]), (points.iloc[j[1]]["lat"], points.iloc[j[1]]["lon"]))
        if (len(a) > 1):
            routes.append(a)
        if (len(b) > 1):
            routes.append(b)
    return routes

In [ ]:
# IMPLEMENTATION OF DIJKSTRAS ALGHORITM
def get_vertex_from_dijkstras(start_index = 0):
    dijkstras_graph = Graph(len(coordinate_data))
    vp = get_vertexes()
    counter = 0
    for i in get_neighbors_of_indexes():
        dijkstras_graph.add_edge(counter, i[0], vp[counter][i[0]])
        dijkstras_graph.add_edge(counter, i[1], vp[counter][i[1]])
        counter += 1
    return dijkstra(dijkstras_graph, start_index)

In [ ]:
def get_closest_vertex_from_dijkstras(inputDict : dict):
    inputDict.pop(0) # First value is always start node itself so it is zero
    smallest_keys = []
    counter = 0
    while len(inputDict) > 0:
        smallest_keys.append(min(inputDict, key=inputDict.get))
        inputDict.pop(smallest_keys[counter])
        counter += 1
    return smallest_keys

In [ ]:
def plot_multi_route_folium():
    routes = []
    route_indexes = []
    dijkstra_path = get_closest_vertex_from_dijkstras(get_vertex_from_dijkstras())
    route_1 = get_shortest_route(coord_tuple_by_index(0), coord_tuple_by_index(dijkstra_path[0]))
    routes.append(route_1)
    route_indexes.append([0, dijkstra_path[0]])
    route_map = ox.plot_route_folium(graph, route_1, route_color='aqua', opacity=0.9,
    popup_attribute='length', tooltip=f"Route between point 0 and {dijkstra_path[0]}")
    
    for i in range(len(dijkstra_path[1:])):
        new_route = get_shortest_route(coord_tuple_by_index(dijkstra_path[i]), coord_tuple_by_index(dijkstra_path[i+1]))
        routes.append(new_route)
        route_indexes.append([dijkstra_path[i], dijkstra_path[i+1]])
        route_map = ox.plot_route_folium(graph, new_route, route_map=route_map, 
        route_color=color_list[i], route_opacity=0.6, 
        popup_attribute='length', tooltip=f"Route between point {dijkstra_path[i]} and {dijkstra_path[i+1]}")
    
    for i in range(len(coordinate_data)):
        folium.Marker(coord_tuple_by_index(i), tooltip = f"{i}. nokta", 
                      icon=folium.Icon(color='blue', icon='home', prefix='fa')).add_to(route_map)
        folium.Circle(coord_tuple_by_index(i), 10).add_to(route_map)
        folium.map.Marker(
        coord_tuple_by_index(i),
        icon=folium.DivIcon(
            icon_size=(60,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 13pt; background-color:{color_list[i+1]};">Ev: {str(i)}</div>',
            )
        ).add_to(route_map)

    for i in range(len(routes)):
        coord = graph.nodes[routes[i][round(len(routes[i])/2)]]
        folium.Marker((coord["y"], coord["x"]), tooltip = f"{i}. nokta",
            icon=folium.Icon(color='green', icon='road', prefix='fa')).add_to(route_map)

        folium.map.Marker(
        (coord["y"], coord["x"]),
        icon=folium.DivIcon(
            icon_size=(90,50),
            html=f'<div style="color:black; font-weight:bold; margin-top:10px;">{route_indexes[i][0]} ve {route_indexes[i][1]} arasındaki yol</div>',
            )
        ).add_to(route_map)
    return route_map

### IMPLEMENT

In [ ]:
pd.DataFrame(get_vertexes()) # Inspect distance between vertexes

,0,1,2,3
0,316.713,198.227,417.123,533.105
1,316.713,444.007,731.259,696.038
2,198.227,444.007,343.488,696.333
3,417.123,731.259,343.488,367.468
4,533.105,696.038,696.333,367.468


In [ ]:
pd.DataFrame(get_neighbors_of_indexes()).T # Inspect closes neighbor vertex indexes

,0,1,2,3,4
0,1,0,0,2,3
1,0,1,2,3,0


In [ ]:
D = get_vertex_from_dijkstras() # Default start index is 0
for vertex in range(len(D)):
    print("Distance from vertex 0 to vertex", vertex, "is", D[vertex])

Distance from vertex 0 to vertex 0 is 0
Distance from vertex 0 to vertex 1 is 316.71299999999997
Distance from vertex 0 to vertex 2 is 198.22699999999998
Distance from vertex 0 to vertex 3 is 541.7149999999999
Distance from vertex 0 to vertex 4 is 533.105


In [ ]:
get_closest_vertex_from_dijkstras(D)

[2, 1, 4, 3]

In [ ]:
plot_multi_route_folium() # Start point is 0